In [3]:
import json, requests, pandas as pd
import datetime

StatementMeta(, 11833a4a-3015-4d8e-b6f6-a7e32ed19466, 5, Finished, Available)

In [4]:
def AuthenticateWithSP():
    key_vault = ""
    tenant = ""
    client = ""
    client_secret = mssparkutils.credentials.getSecret(key_vault , "clientsecret")    
 
    try:
        from azure.identity import ClientSecretCredential
    except Exception:
        !pip install azure.identity
        from azure.identity import ClientSecretCredential
 
    # Generates the access token for the Service Principal
    api = 'https://analysis.windows.net/powerbi/api/.default'
    auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                              tenant_id = tenant,
                              client_id = client,
                              client_secret = client_secret)
    access_token = auth.get_token(api)
    access_token = access_token.token
    header = {'Authorization': f'Bearer {access_token}'}
    print('\nSuccessfully authenticated.')
    return header


StatementMeta(, 11833a4a-3015-4d8e-b6f6-a7e32ed19466, 6, Finished, Available)

In [5]:
header= AuthenticateWithSP()

StatementMeta(, 11833a4a-3015-4d8e-b6f6-a7e32ed19466, 7, Finished, Available)


Successfully authenticated.


In [7]:
#########################################################################################
# Variable definition
#########################################################################################
batch = 0
batchSize = 500
WhileCondition= True
base_url = 'https://api.powerbi.com/v1.0/myorg/'

StatementMeta(, 11833a4a-3015-4d8e-b6f6-a7e32ed19466, 9, Finished, Available)

In [ ]:
while WhileCondition == True:
    skip = batch * batchSize
    API_url = base_url+"admin/groups?$skip="+str(skip)+"&$top="+str(batchSize)+"&$expand=reports,dashboards,datasets,dataflows,users"
    response = requests.get(API_url, headers=header)

    if response.status_code == 429:
        time.sleep(3601)
        header= AuthenticateWithSP()
        response = requests.get(API_url, headers=header)
        
    JSONContent = json.loads(response.content)
    totalWorkspaces= JSONContent['@odata.count']

    if skip < totalWorkspaces:
        if batch == 0:
            print(batch)
            JSONContentExplode = JSONContent['value']
            df_JSONContentExplode=spark.createDataFrame(data=JSONContentExplode)
            batch=batch + 1
        else:
            print(batch)
            JSONContentExplode = JSONContent['value']
            temp=spark.createDataFrame(data=JSONContentExplode, schema=df_JSONContentExplode.schema)
            df_JSONContentExplode=df_JSONContentExplode.unionByName(temp, allowMissingColumns=True)
            batch=batch + 1    
        
    else: 
        WhileCondition= False

In [6]:
df_content=df_JSONContentExplode\
                                 .withColumnRenamed("Id","WorkspaceId")\
                                 .withColumnRenamed("name","Workspace_name")

StatementMeta(, 2e185f2c-25d8-4971-ba10-b87cd1fbfa16, 8, Finished, Available)

In [7]:
#workspaces
from pyspark.sql.types import *
from pyspark.sql.functions import *


Workspaces= df_content\
.select('WorkspaceId','Workspace_name','capacityId','isOnDedicatedCapacity','defaultDatasetStorageFormat','hasWorkspaceLevelSettings ','isReadOnly','type','description','pipelineId','users')\
.withColumnRenamed("hasWorkspaceLevelSettings ","hasWorkspaceLevelSettings")

Workspaces= Workspaces.withColumn("users_admin",expr("filter(users, x -> x['groupUserAccessRight'] = 'Admin')"))\
                       .withColumn("isOrphaned", when(size(col("users_admin"))==0, True).otherwise(False))\
                       .select('WorkspaceId','Workspace_name','capacityId','isOnDedicatedCapacity','defaultDatasetStorageFormat','hasWorkspaceLevelSettings','isReadOnly','type','description','pipelineId','isOrphaned')

Workspaces.write.format('delta').mode("overwrite").option("overwriteSchema", "True").save("Files/Workspaces")

StatementMeta(, 2e185f2c-25d8-4971-ba10-b87cd1fbfa16, 9, Finished, Available)

In [8]:
#dataset workspaces
from pyspark.sql.types import *
from pyspark.sql.functions import *

Workspace_datasets= df_content.select('WorkspaceId',posexplode_outer('datasets').alias('rank','ListItem')).where(col("rank").isNotNull())

keys = (Workspace_datasets
    .select(explode("ListItem"))
    .select("key")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect())

exprs = [col("ListItem").getItem(k).alias(k) for k in keys]

Workspace_datasets= Workspace_datasets.select('WorkspaceId','rank',*exprs)

Workspace_datasets.write.format('delta').mode("overwrite").option("overwriteSchema", "True").save("Files/Workspace_datasets")

StatementMeta(, 2e185f2c-25d8-4971-ba10-b87cd1fbfa16, 10, Finished, Available)

In [9]:
#reporte workspaces
from pyspark.sql.types import *
from pyspark.sql.functions import *

Workspace_reports= df_content.select('WorkspaceId',posexplode_outer('reports').alias('rank','ListItem')).where(col("rank").isNotNull())

keys = (Workspace_reports
    .select(explode("ListItem"))
    .select("key")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect())

exprs = [col("ListItem").getItem(k).alias(k) for k in keys]

Workspace_reports= Workspace_reports.select('WorkspaceId','rank',*exprs)
Workspace_reports.write.format('delta').mode("overwrite").option("overwriteSchema", "True").save("Files/Workspace_reports")

StatementMeta(, 2e185f2c-25d8-4971-ba10-b87cd1fbfa16, 11, Finished, Available)

In [10]:
#workspace dataflows
from pyspark.sql.types import *
from pyspark.sql.functions import *

Workspace_dataflows= df_content.select('WorkspaceId',posexplode_outer('dataflows').alias('rank','ListItem')).where(col("rank").isNotNull())
                                        

keys = (Workspace_dataflows
    .select(explode("ListItem"))
    .select("key")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect())

exprs = [col("ListItem").getItem(k).alias(k) for k in keys]

Workspace_dataflows= Workspace_dataflows.select('WorkspaceId','rank',*exprs)
Workspace_dataflows.write.format('delta').mode("overwrite").option("overwriteSchema", "True").save("Files/Workspace_dataflows")

StatementMeta(, 2e185f2c-25d8-4971-ba10-b87cd1fbfa16, 12, Finished, Available)

In [11]:
#workspace dashboards
from pyspark.sql.types import *
from pyspark.sql.functions import *

Workspace_dashboards= df_content.select('WorkspaceId',posexplode_outer('dashboards').alias('rank','ListItem')).where(col("rank").isNotNull())
                                        

keys = (Workspace_dashboards
    .select(explode("ListItem"))
    .select("key")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect())

exprs = [col("ListItem").getItem(k).alias(k) for k in keys]

Workspace_dashboards= Workspace_dashboards.select('WorkspaceId','rank',*exprs)
Workspace_dashboards.write.format('delta').mode("overwrite").option("overwriteSchema", "True").save("Files/Workspace_dashboards")

StatementMeta(, 2e185f2c-25d8-4971-ba10-b87cd1fbfa16, 13, Finished, Available)

In [ ]:
#workspace users
from pyspark.sql.types import *
from pyspark.sql.functions import *

Workspace_users_groups= df_content.select('WorkspaceId',posexplode_outer('users').alias('rank','ListItem')).where(col("rank").isNotNull())
display(Workspace_users_groups)

keys = (Workspace_users_groups
    .select(explode("ListItem"))
    .select("key")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect())

exprs = [col("ListItem").getItem(k).alias(k) for k in keys]

Workspace_users_groups= Workspace_users_groups.select('WorkspaceId','rank',*exprs)
Workspace_users_groups.write.format('delta').mode("overwrite").option("overwriteSchema", "True").save("Files/Workspace_users_groups")